In [1]:
from typing_extensions import Literal, Annotated 
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph,START,END
from langgraph.types import Command
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
LANGCHAIN_TRACING = os.getenv("LANGCHAIN_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "deepseek-r1-distill-llama-70b")

In [5]:
llm.invoke("Hi")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 4, 'total_tokens': 20, 'completion_time': 0.075977248, 'prompt_time': 0.009812677, 'queue_time': 0.20898452, 'total_time': 0.085789925}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--775c0bf8-6083-436d-aee9-6af0ea39f956-0', usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 20})

In [6]:
def clean_text(text):
    import re
    clean_text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL)
    return clean_text

response = llm.invoke("Hello!<think>this is a thought</think> How are you?")
clean_text(response.content)

"\n\nHello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?"

In [7]:
tavily_tool = TavilySearchResults()

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_55241/583234793.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults()


In [8]:
tavily_tool.invoke("Who is Ronaldo?")

[{'title': 'Cristiano Ronaldo',
  'url': 'https://en.wikipedia.org/wiki/Cristiano_Ronaldo',
  'content': 'Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: (/wiki/Help:IPA/Portuguese "Help:IPA/Portuguese") ⓘ; born 5 February 1985) is a Portuguese professional footballer who plays as a forward "Forward (association football)") for, and captains "Captain (association football)"), both Saudi Pro League club Al-Nassr and the Portugal national team. Nicknamed CR7, he is widely regarded as one of the greatest players in history, and has won numerous individual accolades throughout his [...] One of the world\'s most marketable and famous athletes, Ronaldo was ranked the world\'s highest-paid athlete by Forbes on five occasions, and the world\'s most famous athlete by ESPN from 2016 to 2019. Time "Time (magazine)") included him on their list of the 100 most influential people in the world in 2014. He is the most popular sportsperson on social media: he counts over 1 billion total 

In [9]:
code = """ 
x = 5
y = 4 
result = x + y
print(result ** 2)
"""

In [10]:
@tool 
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """ 
    Use this to execute python code and do math. If you want to see the output of a value, 
    you should print it out with 'print(...)'. This is visible to the user.
    """
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        result = f"Failed to execute code with error: {repr(e)}"

    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str

In [11]:
print(python_repl_tool.invoke(code))

Python REPL can execute arbitrary code. Use with caution.


Successfully executed:
\`\`\`python
 
x = 5
y = 4 
result = x + y
print(result ** 2)

\`\`\`
Stdout: 81



In [12]:
members = ["researcher", "coder"]

In [13]:
options = members + ["FINISH"]

In [14]:
options

['researcher', 'coder', 'FINISH']

In [15]:
class Router(TypedDict):
    """ Worker to route to next agent. If no workers are needed, route to FINISH. """
    next: Literal['researcher', 'coder', 'FINISH']

In [16]:
class State(MessagesState):
    next: str

In [17]:
system_prompt = """ 
You are a supervisor and you are tasked with managing a conversation between the following workers: {members}.
Given the following user request, decide which worker to route to next. If no more workers are needed, route to FINISH.
Each worker will perform their specific task and return their results and status to you. When finished, response with FINISH.
"""

In [18]:
def supervisor_node(state: State) -> Command[Literal["researcher", "coder", "__end__"]]:
    messages = [{"role": "system", "content": system_prompt},] + state["messages"]

    response = llm.with_structured_output(Router).invoke(messages)

    goto = response["next"]
    print(f"Routing to {goto}")

    if goto == "FINISH":
        goto = END
    return Command(goto=goto, update={"next": goto})

In [19]:
def research_node(state: State) -> Command[Literal["supervisor"]]:
    research_agent = create_react_agent(
        llm,
        tools=[tavily_tool],
        prompt="You are a researcher. DO not do any math. Just research the topic and return your findings."
    )
    result = research_agent.invoke(state)

    return Command(
        update = {
            "messages": [HumanMessage(content=result["messages"][-1].content, name="researcher")]
        },
        goto="supervisor",
    )

In [20]:
def code_node(state: State) -> Command[Literal["supervisor"]]:
    code_agent = create_react_agent(
        llm,
        tools=[python_repl_tool],
        prompt="You are a coder. You can do math and write python code. Use python to do any calculations or generate any code."
    )
    result = code_agent.invoke(state)

    return Command(
        update = {
            "messages": [HumanMessage(content=result["messages"][-1].content, name="coder")]
        },
        goto="supervisor",
    )
    

In [21]:
graph = StateGraph(State)

In [22]:
graph.add_node("supervisor", supervisor_node)
graph.add_node("researcher", research_node)
graph.add_node("coder", code_node)

In [23]:
graph.add_edge(START, "supervisor")

In [24]:
app = graph.compile()

In [25]:
# from IPython.display import Image, display

# display(Image(app.get_graph().draw_mermaid_png()))

In [26]:
# for s in app.stream({"messages": [("user", "What's the square root of 42?")]}, subgraphs=True):
#     print(s)
#     print("----")

In [27]:
# app.invoke({"messages": [("user", "What's the square root of 42?")]})

In [28]:
app.invoke({"messages": [("user", "What's double the age of max verstappen??")]})

Routing to researcher
Routing to FINISH


{'messages': [HumanMessage(content="What's double the age of max verstappen??", additional_kwargs={}, response_metadata={}, id='b0f118bb-9ba0-4973-87ba-71ed7e52cf78'),
  HumanMessage(content='Max Verstappen was born on September 30, 1997, which makes him 26 years old as of 2023. Double his age would be 52.', additional_kwargs={}, response_metadata={}, name='researcher', id='a57cf3fe-f443-4d17-a02a-3680eed089cf')],
 'next': '__end__'}